In [1]:
!pip install tabulate

In [2]:
!pip install "colorama>=0.3.8"

In [3]:
!pip install future

In [4]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 123.6MB 158kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.3-py2.py3-none-any.whl size=123649955 sha256=e00394da57a4c77e66208975752b3382a24887535ec9a3185b9d278c731b1530
  Stored in directory: /root/.cache/pip/wheels/4a/a6/8d/0880c51d5aa9746a87f61d9b66d8a98e7b9cf400c81ffc944e
Successfully built h2o


In [0]:
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

In [16]:
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,7 mins 35 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,1 day
H2O cluster name:,H2O_from_python_unknownUser_hgdz38
H2O cluster total nodes:,1
H2O cluster free memory:,2.930 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [7]:
# Import a sample binary outcome dataset into H2O
data = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
# Identify predictors and response
x = data.columns
y = "response"
x.remove(y)

# For binary classification, response should be a factor
data[y] = data[y].asfactor()
test[y] = test[y].asfactor()

# Split data into train & validation
ss = data.split_frame(seed = 1)
train = ss[0]
valid = ss[1]


In [12]:
print(data[y])

response
1
1
1
0
1
0
1
1
1
1


In [0]:
# GBM hyperparameters
gbm_params1 = {'learn_rate': [0.01, 0.1],
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.2, 0.5, 1.0]}


In [17]:
# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)



gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [18]:
gbm_grid1.train(x=x, y=y,
                training_frame=train,
                validation_frame=valid,
                ntrees=100,
                seed=1)

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [19]:
# Get the grid results, sorted by validation AUC
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='auc', decreasing=True)
gbm_gridperf1

     col_sample_rate  ...           model_ids                 auc
0                1.0  ...  gbm_grid1_model_24  0.7901280555015237
1                1.0  ...   gbm_grid1_model_6  0.7892806198534656
2                0.5  ...  gbm_grid1_model_29  0.7883284704661869
3                1.0  ...  gbm_grid1_model_30   0.788159891071776
4                0.5  ...  gbm_grid1_model_35  0.7861152175322571
5                0.5  ...  gbm_grid1_model_23  0.7859936458535953
6                0.5  ...  gbm_grid1_model_11   0.785658432211632
7                0.5  ...  gbm_grid1_model_14  0.7844479024833042
8                1.0  ...  gbm_grid1_model_12  0.7839048823186151
9                0.2  ...  gbm_grid1_model_28  0.7837774751993776
10               0.5  ...   gbm_grid1_model_5  0.7836383971989885
11               0.5  ...  gbm_grid1_model_32  0.7832966997341633
12               0.2  ...  gbm_grid1_model_31  0.7822521558711015
13               1.0  ...  gbm_grid1_model_15  0.7806461129481943
14        

In [0]:
# Grab the top GBM model, chosen by validation AUC
best_gbm1 = gbm_gridperf1.models[0]

In [21]:

# Now let's evaluate the model performance on a test set
# so we get an honest estimate of top model performance
best_gbm_perf1 = best_gbm1.model_performance(test)

best_gbm_perf1.auc()


0.7781788272580651